# Solidity : Control, DataStruct

### 반복의 비용

횟수    |    sum을 로컬에   |    sum을 state variable에

10회 반복 =  30,000 = 64,000

100회 반복 =  78,000  = 368,000 

## -----------------------------------------------------------------------------

## 실습: 반복문, 조건문 사용

In [4]:
%%writefile src/LoopTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract LoopTest {
    uint sum;
    constructor() public{
        init();
    }
    function forLoop() public {
        //init();
        uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            sumLocal += i;
        }
        sum = sumLocal;
    }
    function forLoopCostly() public {
        init();
        //uint sumLocal = 0;
        for(uint i = 0; i < 101; i++) {
            //sumLocal += i;
            sum += i;
        }
        //sum = sumLocal;
    }
    function whileLoop() public {
        uint i = 0;
        uint sumLocal = 0;
        while(i <= 100) {
            sumLocal += i;
            i++;
        }
        sum = sumLocal;
    }
    function whileLoopBreak() public {
        uint sumLocal = 0;
        uint i = 0;
        while(true) {
            sumLocal += i;
            if(i==100) break;
            i++;
        }
        sum = sumLocal;
    }   
    function doWhileLoop() public {
        uint i = 0;
        uint sumLocal = 0;
        do {
            sumLocal += i;
            i++;
        } while(i <= 100);
        sum = sumLocal;
    }
    function init() public { sum = 0; }
    function getSum() view public returns(uint) { return sum; }
}

Overwriting src/LoopTest.sol


In [5]:
!solc src/LoopTest.sol

Compiler run successful, no output requested.


## -----------------------------------------------------------------------------

## 조건문

In [6]:
%%writefile src/ConditionalTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract ConditionalTest {
    function toLowerCase(string memory str) pure public returns (string memory) {
        bytes memory bIn = bytes(str);
        bytes memory bOut = new bytes(bIn.length);
        uint8 _iIn = 0;
        for (uint i = 0; i < bIn.length; i++) {
            _iIn = uint8(bIn[i]);
            if (_iIn >= 65 && _iIn <= 90)    // ascii A:65 ~ Z:90                
                bOut[i] = bytes1(_iIn + 32); // ascii a:97 ~ z:122
            else 
                bOut[i] = bIn[i];
        }
        return string(bOut);
    }
    function abs(int i) pure public returns (int) {
        return (i > 0) ? i : -i;
    }
}

Writing src/ConditionalTest.sol


In [7]:
!solc src/ConditionalTest.sol

Compiler run successful, no output requested.


## -----------------------------------------------------------------------------

## 실습 : 인사하는 컨트랙
modifier를 사용하면 조건문을 사용하지 않을 수 있다. 조건문에서 비교문을 사용하게 되는데, string은 참조타입이라 비교를 하면 주소를 비교하게 된다

## 1단계 컨트랙 개발

In [8]:
%%writefile src/Hello1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Hello1 {
    string private hello;
    address private owner;
    event PrintLog(address addr, string s);

    constructor(string memory _hello) public {
        hello = _hello;
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    modifier isOwner() {
        if (msg.sender != owner) {
            revert();
        }
        _; //insert here the calling function
    }
    function setHello() public {
        string memory s = "";
        if (msg.sender == owner) {
            s = "Hello";
        } else {
            s = "Olleh";
        }
        emit PrintLog(msg.sender, s);
        hello = s;
    }
    function compareTo(string memory _str) view public returns(bool) {
        bool isEqual = false;
        //if (hello == _str) {
        if (keccak256(bytes(hello)) == keccak256(bytes(_str))) {
            isEqual = true;
        }
        return isEqual;
    }
    function getBalance() view public isOwner returns(uint) {
        return address(this).balance;
    }
}

Writing src/Hello1.sol


## 2단계 컴파일

In [9]:
!solc src/Hello1.sol --combined-json abi > src/Hello1ABI.json

In [10]:
!solc src/Hello1.sol --combined-json bin > src/Hello1BIN.json

## 3단계 배포

In [15]:
%%writefile src/Hello1DeployFromFile.js
var Web3=require('web3');
var _abiJson = require('./Hello1ABI.json');
var _binJson = require('./Hello1BIN.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiJson.contracts); // reading ['src/Hello1.sol:Hello1']

_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin="0x"+_binJson.contracts[contractName].bin;  // ok with "0x"

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin, arguments: ["Hello from web3"]})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Hello1DeployFromFile.js


In [16]:
!node src/Hello1DeployFromFile.js

Deploying the contract from 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
hash: 0xfecb7cb7cfd4d1f027088cc7c11ce20f5e3077a24b8913ca4fa35bd2c9d75310
---> The contract deployed to: 0x0F5a9529F1FcFd4342D81feD970C5ed855A99101


## 4단계 사용
이벤트는 transaction log에 저장이 된다. 이벤트는 거래가 마이닝되어 블럭에 포함되어야만 발생된다. 트랙잭션이 pending되어 완료가 되지 않으면, 로그에 포함되지 않기 때문에 여기서 이벤트가 발생하는지 지켜보는 경우 대기하게 된다. 단 이벤트가 발생하는 순서는 지켜진다. 단순하게 블록체인의 transaction이 아닌 call()은 이벤트를 발생하지 않게 된다.

In [21]:
%%writefile src/Hello1UseFromFile.js
var Web3=require('web3');
var _abiJson = require('./Hello1ABI.json');

var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));  //ok

contractName=Object.keys(_abiJson.contracts); // reading ['src/Hello1.sol:Hello1']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0x0F5a9529F1FcFd4342D81feD970C5ed855A99101");
    var event = hello.events.PrintLog(function (error, event) {
        console.log(">>> Event fired: " + JSON.stringify(event.returnValues));
    })
    .on('>> data', function(event) {
        console.log(event);
    })
    .on('>> changed', function(event) {
        console.log(event);
    })
    .on('>> error', console.error);
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello().send({from: accounts[0]});
    hello.methods.sayHello().call().then(console.log);
    hello.methods.compareTo("Hello").call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    
}
doIt()

Overwriting src/Hello1UseFromFile.js


* 웹소켓을 사용하면, 프로세스가 계속되고 있다. 프로세스를 인위적으로 해제해주어야 한다.

In [22]:
!node src/Hello1UseFromFile.js

^C


## -----------------------------------------------------------------------------

## 실습 : 블록체인에서 컨트랙 삭제
블록체인에서는 영구히 남고 직접 삭제 할 수 없으므로 메모리에서 해체시켜야 한다.
```
selfdestruct()
selfdestruct(payable(owner));
```
메모리 해제시 문제점이있다. 컨트랙에서 메모리를 해제하면 잔고의 처리 문제가 굉장히 중요해진다.

그래서 selfdesturct()할 때는 계좌를 무조건 적어줘야한다. 즉, 잔고를 받을 수 있는 주소를 넣어줘야 한다.

## 1단계 컨트랙 개발

In [25]:
%%writefile src/Hello2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Hello2 {
    string hello;
    address owner; //address payable owner;

    event PrintLog(string _s);
    constructor() public { 
        owner = msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    function setHello(string memory _hello) public payable {
        hello = _hello;
        emit PrintLog(_hello);
    }
    function getBalance() view public returns(uint) {
        return address(this).balance;
    }
    function kill() public {
        if (msg.sender == owner) selfdestruct(payable(owner));
    }
}

Overwriting src/Hello2.sol


* 23: 우리가 원하는(지우고자 하는) 함수, 지우는 권한을 최초의 주소가 같으면 자신한테 잔고를 전달한다. 

## 2단계 컴파일

In [26]:
!solc src/Hello2.sol --combined-json abi > src/Hello2ABI.json

In [27]:
!type src\Hello2ABI.json

{"contracts":{"src/Hello2.sol:Hello2":{"abi":"[{\"inputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"internalType\":\"string\",\"name\":\"_s\",\"type\":\"string\"}],\"name\":\"PrintLog\",\"type\":\"event\"},{\"inputs\":[],\"name\":\"getBalance\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"kill\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"sayHello\",\"outputs\":[{\"internalType\":\"string\",\"name\":\"\",\"type\":\"string\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[{\"internalType\":\"string\",\"name\":\"_hello\",\"type\":\"string\"}],\"name\":\"setHello\",\"outputs\":[],\"stateMutability\":\"payable\",\"type\":\"function\"}]"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


In [28]:
!solc src/Hello2.sol --combined-json bin > src/Hello2BIN.json

## 3단계 배포


In [29]:
%%writefile src/Hello2DeployFromFile.js
var Web3=require('web3');
var _abiJson = require('./Hello2ABI.json');
var _binJson = require('./Hello2BIN.json');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

contractName=Object.keys(_abiJson.contracts); // reading ['src/Hello2.sol:Hello2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
//_abiArray=_abiJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin="0x"+_binJson.contracts[contractName].bin; //ok without "0x"

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Writing src/Hello2DeployFromFile.js


In [30]:
!node src/Hello2DeployFromFile.js

- contract name:  src/Hello2.sol:Hello2
Deploying the contract from 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
hash: 0x2ecc73be97d478a725ec9233dc630e86bf828885e17e2c66664f8c35cea06180
---> The contract deployed to: 0xbb780C8053a37F9dC38bE9DeE61463757C88dD71


## 4단계 사용

In [37]:
%%writefile src/Hello2UseFromFile.js
var Web3=require('web3');
var _abiJson = require('./Hello2ABI.json');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));  //ok

contractName=Object.keys(_abiJson.contracts); // reading ['src/Hello2.sol:Hello2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0xbb780C8053a37F9dC38bE9DeE61463757C88dD71");
    var event = hello.events.PrintLog(function (error, result) {
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result.returnValues));
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    hello.methods.sayHello().call().then(console.log);  //null
    await hello.methods.setHello("Hello World!").send({from: accounts[0], value: 1111});
    hello.methods.sayHello().call().then(console.log);
    hello.methods.getBalance().call(function(err, bal) { console.log("Contract Balance: "+bal) });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1);  //force exit
    hello.methods.kill().send({from: accounts[0]})
}

doIt()

Overwriting src/Hello2UseFromFile.js


In [38]:
!node src/Hello2UseFromFile.js

- contract name:  src/Hello2.sol:Hello2
Account: 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
Balance before: 999996376887999995356
Hello World!
Event fired: {"0":"Hello World!","_s":"Hello World!"}
Balance after: 999996319093999994245
Balance diff: 57793999929344


# --------------------------------------------------------------

# 1.4 무작위 수의 생성
1. 블록체인에서는 무작위 수를 생성하는 것은 어렵다. 라이브러리가 내장되어 있지 않고 불러다 쓰면 매우 불편하다. 즉, 라이브러리가 있지 않다. (단점)

2. seed번호를 다르게 선언해주면 무작위 생성이 가능하다. 그렇다면 seed번호를 무엇을 쓸까? 블록체인은 바이트코드로 되어 있기 때문에 쉽지가 않다.

3. 약간의 트릭을 써서 카운터 값을 더해주도록 한다.

In [41]:
%%writefile src/Random.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Random {
    function rand() public view returns(bytes32) {
        return keccak256(abi.encodePacked(block.timestamp, block.difficulty));
    }
    function rand0and250() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%251);
    }
    function rand0and9() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%10);
    }
    function rand0and2() public view returns(uint8) {
        return uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
    }
    function genRandomInteger() view public returns(uint8[] memory) {
        uint8[] memory r = new uint8[](10);
        for(uint i = 0; i<r.length; i++)
            r[i] = uint8(uint256(keccak256(abi.encodePacked(block.timestamp + i, block.difficulty)))%10);
        return r;
    }
}

Writing src/Random.sol


* 7: abi.encodePacked ~~는 컴퓨터의 시간을 가져온다.(timestamp), difficulty도 매번 변동이 가능하다. 다른것으로 시드를 써도 되는데 매번 변동을 하는 이 시드 2개를 이용한다. keccak256: 256비트로 해쉬코드를 만든다. 이건 길이가 굉장이 큰 경우다. 저 해쉬를 정수로 형변환하고 범위를 modul로 나눠주면 원하는 범위로 랜덤값을 구할 수 있다.
* 10 : 0~250, uint 8비트로 나눠주고 251로 모듈러스를 만들어 나눠준다.
* 16 : 0~2까지면 모듈러스를 %3을 해준다.
* 19 : 시드를 변경해주지 않으면 매번 똑같은 랜덤숫자가 나온다. 난스 nouce를 매번 만들어준다. 즉, i라는 넌스를 계속 더해줘서 만들어주면 (23 : + i 부분).. 
* 배열 형식 저장은 메모리를 사용하므로 memory를 사용한다. 그래서 memory를 적어준다.
* 함수는 메모리에 만들어지므로 메모리를 쓴다.

In [42]:
!solc src/Random.sol

Compiler run successful, no output requested.


# --------------------------------------------------------------

# 2. 데이터구조
## 2.1 배열
### 2.1.1 고정배열
함수 내의 모든 것은 메모리를 쓴다.
```
매개변수로 넘겨주는 것 (참조일 때)
선언 하는 것
반환할 때 (참조)
```
함수 밖에서는 스토리지를 쓴다.

고정배열에서는 push를 사용할 수 없다.

#### 데이터타입의 범위를 벗어나는 값을 지정하지 않는다
```
uint8[5] balance = [255, 255, 95, 50, -1];  //오류: -1이 음수 이경우 uint8은 0~255이므로 오류 발 생
uint8[5] balance = [255, 256, 95, 50, 1];  //오류: 256이 8비트 범위 밖 -> uint16
uint16[5] balance = [255, 65536, 95, 50, 1];  //오류: 65536이 16비트 범위 밖 -> uint24
```
#### 함수에서 고정배열을 초기화
```
mathMarks=[100,60,95,50,80]; //오류: 함수 내 우측 배열은 uint8 memory -> mathMarks의 int로 변환 오류
우측은 매모리, 좌측은 스토리지, 따라서 안된다. (다른 언어는 가능) 
mathMarks=uint8([100,60,95,50,80]); //오류: 우측 배열 uint8 memory -> uint8() 형변환 오류
```
따라서
```
반복문으로 하나씩 읽어서 uint8 -> int8 -> int로 변환해 주어야 한다
contrat ArrayTest2 {
  int[5] mathMarks;  //storage 변수
  function setMathMarks() public {
      uint8[5] memory temp = [100, 60, 95, 50, 80];
      for (uint i = 0; i < temp.length; i++)
          mathMarks[i] = int(int8(temp[i])); //ok: uint8 -> int8 -> int256
  }
}
uint8 [] -> uint 256 -> int 256 이런 방식으로 바꿔줘야한다. 메모리 하나하나가 돈이기 때문에 이렇게 한다.
또 한거번에 못 넣어준다. for문으로 하나하나씩 넣어줘야 한다.
```
#### 배열 storage의 읽기
```
storage 배열을 리턴하는 것이 안된다. 왜? 반환할때 배열은 메모리가 된다. 즉, 스토리지 -> 메모리가 불가능하므로 형변환이 필요하다.
```
## 동적배열
함수 내에서 사용할 수 없다. 메모리만 가능.
```
contract my { 
    uint[] myArr; //동적배열 가능.
    function setArr() public {
       myArr.push(11); //동적배열에 대해 push 함수를 사용한다.
    }
}
contract my { 
    function setArr() public {
       uint[] myArr; //오류. 로컬에서 동적배열 사용할 수 없다.
    }
}
```
아래는 아무것도 안적으면 스토리지 이므로 아래는 오류.

push는 storage 배열에만 사용하고 (즉 상태변수의 배열), memory 배열에는 사용할 수 없다.
### 동적배열을 memory에서 사용할 때 new, 괄호 안에 개수 적는다.
```
function setLocalDynamicArr() view public returns(uint) {
    //uint[] myArr; //오류. storage 또는 memory 선언 필요 (안적으면 스토리지)
    //uint[] storage myArr; //오류. uninitialized storage (그렇다고 적어주면 함수 내에선 못씀)
    //uint[] memory myArr; //오류. 동적배열 선언 가능. 그러나 push 못함. (메모리니까 선언이 가능, 메모리는 push 불가능)
    //uint[] myArr = new uint[](3); //오류. storage 또는 memory 필요. 먼갈 안적어줌
    //uint[] storage myArr = new uint[](3); //오류. 좌측은 memory. 우측 stoarge로 변환 오류
    uint[] memory myArr = new uint[](3); //정상. memory 동적배열은 크기를 할당해야 한다. 우측에서 이렇게 만들면 무조건 메모리다. 함수내에서 선언하는 것은 무조건 메모리 이므로! 이 경우는 맞다.
    //myArr.push(11); //오류. myArr 크기가 정해져서, 고정배열에 push 사용하지 못함.
    myArr[0]=11; //정상. 인덱스를 사용해서 입력
    myArr[1]=12; //정상.
    //myArr.push(13); //오류.
    //myArr[5]=15; //오류.
    return myArr.length; //반환 3
}
```

### 다차원 배열
```
uint[3][] marks=[[100, 80, 95],[20,30,40]];
```

# 배열 실습
## 1단계 컨트랙 개발

In [48]:
%%writefile src/ArrayTest2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;
pragma experimental ABIEncoderV2;

contract ArrayTest2 {
    string[3] cities1=["Seoul", "Sydney", "Tokyo"];
    string[] cities2 = new string[](2);
    string[] cities3;
    uint24[5] balance = [255, 65536, 95, 50, 1];
    int[5] mathMarks;
    uint[3][] marks=[[100, 80, 95],[20,30,40]];

    function setLocalDynamicArr() pure public returns(uint) {
        uint[] memory myArr = new uint[](3);
        myArr[0]=11;
        myArr[1]=12;
        return myArr.length;
    }
    function getDynamicArrMemory() pure public returns(uint[] memory) {
        uint[] memory num=new uint[](3);  //dynamic
        for (uint i=0; i<num.length; i++)
            num[i]=i;       //push() not allowed for array memeory
        return num;
    }
    function getStringDynamicArrMemory() pure public {
        string[2] memory places = ["9000", "Sydney"];
        places[0]="Seoul";
    }
    
    /*returning 'string[] storage' is not allowed
    function getCities1_() view public returns(string[] memory) {
        return cities1;  //can not return stoarge var. 
    }*/
    
    function getCities1() view public returns(string memory) {
        return cities1[0];
    }
    function getCities1Length() view public returns(uint) { return cities1.length; }
    function setCities23() public {
        string memory my = "seoul";
        cities2[0]="New York";
        cities2.push(my);
        cities2.push("Busan");
        cities3.push("New York");
        cities3.push("Beijing");
    }

    function getCities2() view public returns(string[] memory){
        return cities2;
    }
    function setMathMarks() public {
        uint8[5] memory temp = [100, 60, 95, 50, 80];
        for (uint i = 0; i < temp.length; i++) {
            mathMarks[i] = int(int8(temp[i])); //ok: uint8 -> int8 -> int256
        }
    }
    function getMathMarks() view public returns (int[] memory) {
        int[] memory _arr256 = new int[](mathMarks.length);
        for (uint i = 0; i < mathMarks.length; i++) {
            _arr256[i] = int256(mathMarks[i]);
        }
        return _arr256;
    }
    function getMathAbove70_() view public returns(int[] memory) {
        int[] memory mathAbove70;
        uint counter = 0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) {
                mathAbove70[counter] = mathMarks[i];
                counter++;
            }
        return mathAbove70;
    }
    //run setMathMarks() beforehand
    function getMathAbove70() view public returns(int[] memory) {
        uint8 counter=0;
        uint8 lengthOfMathAbove70=0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) counter++;
        lengthOfMathAbove70=counter;
        int[] memory mathAbove70=new int[](lengthOfMathAbove70);
        counter=0;
        for(uint i=0;i<mathMarks.length;i++) {
            if(mathMarks[i]>70) {
                mathAbove70[counter]=mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }
    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
}

Overwriting src/ArrayTest2.sol


## 2단계 컴파일

In [49]:
!solc src/ArrayTest2.sol --combined-json abi > src/ArrayTest2ABI.json

In [50]:
!solc src/ArrayTest2.sol --combined-json bin > src/ArrayTest2BIN.json

## 3단계 컨트랙 배포
* estimateGas

In [51]:
%%writefile src/ArrayTest2Deploy.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var _binJson = require('./ArrayTest2BIN.json');

var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);

_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!
_bin="0x"+_binJson.contracts[contractName].bin; //ok without "0x"

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);

    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1621137}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address);
}

deploy()

Overwriting src/ArrayTest2Deploy.js


In [52]:
!node src/ArrayTest2Deploy.js

- contract name:  src/ArrayTest2.sol:ArrayTest2
Deploying the contract from 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
hash: 0xe6c6fbe33900f880e11ad6bbaae8ef34c11886fe74cf2fa68d5bd28621b27334
---> The contract deployed to: 0x2eB3932BfA9471873Aa07250fCC3CaD81c8b7997
>> gas: 1353625


## 4단계 사용 

In [54]:
%%writefile src/ArrayTest2Use.js
var Web3=require('web3');
var _abiJson = require('./ArrayTest2ABI.json');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));       //ok

contractName=Object.keys(_abiJson.contracts); // reading ['src/ArrayTest2.sol:ArrayTest2']
console.log("- contract name: ", contractName[0]); //or console.log(contractName);
//_abiArray=_abiJson.contracts[contractName].abi;
_abiArray=JSON.parse(_abiJson.contracts[contractName].abi);    //JSON parsing needed!!

async function doIt() {
    var arr = new web3.eth.Contract(_abiArray, "0x2eB3932BfA9471873Aa07250fCC3CaD81c8b7997");
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    
    arr.methods.setCities23().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    //await arr.methods.setCities23().send({from: accounts[0]});   //out of gas error
    await arr.methods.setCities23().send({from: accounts[0], gas:1353625});
    arr.methods.getCities2().call().then(console.log);

    arr.methods.setMathMarks().estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });

    await arr.methods.setMathMarks().send({from: accounts[0], gas: 162354});
    //ERROR invalid opcode arr.methods.getMathAbove70().call().then(console.log);
    arr.methods.getMarksArr().call().then(console.log);

    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/ArrayTest2Use.js


* 7: 나중에 컨트랙이 여러개 일때 더 넣어줄 수 있다.

In [55]:
!node src/ArrayTest2Use.js

- contract name:  src/ArrayTest2.sol:ArrayTest2
Account: 0x38B90A15f126760Ccd539D9A9fbb8c10bd37c02b
Balance before: 999993496255999992023
>> gas: 161934
[ 'New York', '', 'seoul', 'Busan' ]
>> gas: 132832
Balance after: 999992906723999992023
Balance diff: 589532000026624
[ [ '100', '80', '95' ], [ '20', '30', '40' ] ]


## 실습: Members 배열
```
배열을 검색하는 것은 비용이 많이 발생한다. mapping을 사용하거나, 클라이언트 측에서 배열 데이터를 검색하는 편으로 하자.

문자열비교는keccak256() 해싱을 한 후 비교해야 한다. 아래와 같이 문자열을 해싱한후 ==로 비교를 해야 한다.

keccak256(abi.encodePacked(문자열))==keccak256(abi.encodePacked(찾는 문자열))
```
### 원하는 데이터 찾기


In [56]:
%%writefile src/Members.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Members{
    address owner;
    event printAddr(address arg);
    struct Member{
        uint id;
        string name;
    }
    Member[] public memberArr;
    constructor() public { 
        owner=msg.sender;
    }
    function del() public {
        delete memberArr;
    }
    function delOne(uint i) public{
        delete memberArr[i];  //try pop()
    }
    function add(uint id,string memory name) public {
        memberArr.push(Member(id,name));
    }
    //return Member
    function getMember(string memory who) view public returns(uint, string memory) {
        uint _id;
        string memory _name;
        for(uint i=0;i<memberArr.length;i++) {
            _name=memberArr[i].name;
            if(keccak256(abi.encodePacked(_name))==keccak256(abi.encodePacked(who))) {
                _id=memberArr[i].id;
                _name=memberArr[i].name;
            }
        }
        return (_id,_name);
    }
    function compareStr(string memory s1, string memory s2) pure public returns(bool) {
        return keccak256(abi.encodePacked(s1))==keccak256(abi.encodePacked(s2));
    }
    function compareBytes(bytes memory b1, bytes memory b2) pure public returns(bool) {
        return keccak256(b1) == keccak256(b2);
    }
    function getLength() view public returns(uint) {
        return memberArr.length;
    }
}

Writing src/Members.sol


* 31: 인코딩하고 비교하는 과정
* 36: web3에서 받으므로 값 2개를 리턴해야한다.
* 38: 비교함수 만들어보기

In [57]:
!solc src/Members.sol

Compiler run successful, no output requested.


# --------------------------------------------------------------

# 2.2 mapping

## 2.2.1 키와 값 쌍으로 저장

## 실습: 은행 잔고의 mapping

In [60]:
%%writefile src/BankV4.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

//check for Coin.sol (ch12) and BankV3.sol 201906
contract BankV4 {
    address owner;
    mapping (address => uint) balanceOf;
    constructor() public {
        owner = msg.sender;
    }
    // save to individual addresses
    function deposit(uint amount) payable public onlyOwner {
        require(msg.value == amount);
        balanceOf[msg.sender] += amount;
    }
    // forward from owner to another
    function forwardTo(address receiver, uint amount) payable public onlyOwner {
        require(balanceOf[msg.sender] >= amount);
        balanceOf[msg.sender] -= amount; // Subtract from the sender
        balanceOf[receiver] += amount;   // Add the same to the recipient
    }
    function withdraw(address payable receiver, uint amount) public onlyOwner {
        require(balanceOf[receiver] >= amount && address(this).balance >= amount);
        balanceOf[receiver] -= amount;
        receiver.transfer(amount);
    }
    function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balanceOf[owner]);
    }
    function getBalanceOf(address addr) public view returns (uint) {
        return balanceOf[addr];
    }
    modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
}

Overwriting src/BankV4.sol


* 잔고 2가지를 추적하고 있다. 수동적, 자동적
* 8: 없어도 되지만 갑돌이 갑순이 따로따로 계좌를 알려면 선언해줘야한다.
* 9: constructor() 가시성의 종류는 1.public , 2.private, 3.internal, 4.external 
* internal : 블로게인 내에서만 쓰므로 web3에서 부를 수 없다.
* 10: msg. 전역변수 즉, 어디서나 쓸 수 있다. owner은 하느님 신급
* 26: 실제 잔고   
* 29: 수동잔고, 자동잔고 2개

In [61]:
!solc src/BankV4.sol

Compiler run successful, no output requested.


# --------------------------------------------------------------

## 2.2.2 중복키 확인
맵은 그 특성상 중복키는 허용되지 않는다.
```
mapping(address=>Member) memberMap;
struct Member {
    uint id;
    string name;
    bool isMember;
}
if (memberMap(<address>).isMember) 중복키 존재함
```
호출 방식은 if~~ 
##### 삭제
키삭제는 쉽지 않다. 다른 언어에서는 remove()함수가 지원되어, 간편하게 키를 지울 수 있는 것과 비교된다.

delete pendingAccessRequest[key];

## 2.2.3 bidirectional map
주소에 대해 회원(아이디, 이름)을 저장하는 map이다. 이 경우 'id' 또는 '이름'으로 '주소'를 검색하는 기능은 bidiretional map으로 구현하였다. 이렇게 하면, 반복문을 사용하지 않고 검색하는 기능이 가능해 진다. 특정 id에 해당하는 주소를 찾고, 그 주소에 해당하는 값을 읽어올 수 있게 된다.

In [62]:
%%writefile src/MembersMap.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract MembersMap {
    struct Member {
        uint id;
        string name;
    }
    //bidrectional map
    mapping(address=>Member) memberMap;
    mapping(uint=>address) addressById;
    mapping(string=>address) addressByName;
    function addMember (uint _id, string memory _name) public {
       Member memory m=Member(_id, _name);
       memberMap[msg.sender]=m;
       //saving into a bidiretional map to get address by id
       addressById[_id]=msg.sender;
       //saving into a bidiretional map to get address by name
       addressByName[_name]=msg.sender;
    }
    //get Member by id
    function getMemberById(uint _id) view public returns(uint, string memory) {
        Member memory my = memberMap[addressById[_id]];
        return (my.id, my.name);
    }
    // get address (not Member) by name
    function getMemberAddressByName(string memory _name) public view returns(address) {
        return addressByName[_name];
    }
    function getMember(address addr) view public returns (uint, string memory) {
        Member memory m=memberMap[addr];
        return (m.id, m.name);
    }
}

Writing src/MembersMap.sol


* 11: 주소의 Member struct를 조회하는 map
* 12: 이름의 주소를 조회하는 map
* 13~21: Member struct을 추가하고 getMemberByName() 함수에서 이름을 입력하면 주소를 조회할 수 있도록 저장. name이 중복되는 경우, 엎어쓰기 때문에 문제가 될 수 있다. 실제는 중복이 없는 id를 사용하여 bidirectional map을 구현해야 한다.
* 22~24: bidirectional map에서 id에 해당하는 주소를 조회하고, 그 주소키를 이용 Member를 조회
* 27~30: bidirectional map에서 name에 해당하는 주소를 조회
+ 31~34: 주소에 해당하는 Member struct을 조회. (1) addressById[정수]를 하면 주소를 얻을 수 있다. (2) 그러면 memberMap[주소]는 Member라는 값을 얻을 수 있다.

In [63]:
!solc src/MembersMap.sol

Compiler run successful, no output requested.


## 예외

In [64]:
%%writefile src/ExceptionTest.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract ExceptionTest {
    address owner;
    constructor() public {
        owner=msg.sender;
    }
    function requireException() view public returns(string memory) {
        //if (msg.sender != owner) { throw; } // not this way to throw an exception
        require(msg.sender != owner, "Sorry! You are owner. Require failed...");
        return "require condition met";
    }
    function assertException() view public returns(string memory) {
        assert(msg.sender == owner);
        return "asserted";  //do this line only 'assert' is succeeded
    }
    function revertException() view public returns(string memory) {
        if(msg.sender != owner) {
            revert("Sorry! You are NOT owner. Reverted...");
            //return "reverted";  // this line can't be reached if revert is executed 
        } else {
            return "not reverted";
        }
    }
    function raiseException() pure public {
        int x=0;
        int y=0;
        x/y;    //divide by zero
    }
}

Writing src/ExceptionTest.sol


In [65]:
!solc src/ExceptionTest.sol

Compiler run successful, no output requested.


### REMIX 에서
* assertException(누를 때) = 제대로 실행되면 asserted가 실행
* asserted안되게하려면 위에서 다른 계정을 누르고 다시 눌러본다. exception 발생!
* 실제로 보면 잔고가 그대로다.
* raiseException() 누르면 안된다. 막으려면 require해준다. 계정을 바꿔줘야 출력이 안된다. 